# Normalization

- Correcting spelling using model https://huggingface.co/oliverguhr/spelling-correction-english-base
    - this produces too many mistakes
- expanding contractions
- removing punctuation & capitalization


In [1]:
import sqlite3

import pandas as pd

conn = sqlite3.connect("../../../giicg.db")
prompts = pd.read_sql("Select * from manually_corrected_prompts", conn)
prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
756,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code. I want to: Get nodes and edge...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en
757,1845,37,user,\n nun möchte ich judgement balancing m...,Now I want to bring judgement balancing into t...,,,Woman (cisgender),29,de
758,1847,37,user,\n ich sehe keine veränderung im Plot. Was ...,I don't see any change in the plot.,,,Woman (cisgender),29,de
759,1849,2,user,\n I am working on the problem of reconstru...,\n I am working on the problem of reconstru...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en


# Normalization Steps

In [2]:
from helpers.normalization import expand_contractions

prompts['conversational'] = prompts['conversational'].apply(expand_contractions)
prompts


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
756,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code. I want to: Get nodes and edge...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en
757,1845,37,user,\n nun möchte ich judgement balancing m...,Now I want to bring judgement balancing into t...,,,Woman (cisgender),29,de
758,1847,37,user,\n ich sehe keine veränderung im Plot. Was ...,I do not see any change in the plot.,,,Woman (cisgender),29,de
759,1849,2,user,\n I am working on the problem of reconstru...,\n I am working on the problem of reconstru...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en


### Spell Correction Bad Examples

In [3]:
from helpers.normalization import load_spell_correction_model, correct_spelling

tokenizer, model, device = load_spell_correction_model()
text = "hey, I\'m writing an application. Because I want to include some sound snippets, I decided to send them as an application.html file, a style.css and a folder with resources (sound, img, font). Please place the .html and .css document in such a way that it makes sense for this purpose. The application.html file should be structured quite simply - no header, only a body that contains the headline and a flow text. In between then audioplayer and at the end images. It should remember a classic .pdf document -- just interactive thanks to html5."

#text = (expand_contractions(text))
#print(text)
#print(correct_spelling(text, tokenizer, model, device))

#text = "i load it like this:"
#text = (expand_contractions(text))
#print(text)
#print(correct_spelling(text, tokenizer, model, device))

#text = "ok nice. can i let the color switch already on hover? i.e. when im on the work page, and move hover over contact, then work becomes 3e3e3e and contact becomes blue"
#text = (expand_contractions(text))
#print(text)
#print(correct_spelling(text, tokenizer, model, device))

text = "inside a <figure>?"
text = (expand_contractions(text))
print(text)
print(correct_spelling(text, tokenizer, model, device))

model loaded
inside a <figure>?
Inside a computer, what?


In [4]:
from helpers.normalization import remove_punct_cap

prompts_punct_cap = prompts.copy()
prompts_punct_cap['conversational'] = prompts_punct_cap['conversational'].apply(remove_punct_cap)
prompts_punct_cap

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...",parsing data from python iterator how it could...,,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,write shortest tutorial on creating rag on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is faiss,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,transform given code to process large mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
756,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code i want to get nodes and edges ...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en
757,1845,37,user,\n nun möchte ich judgement balancing m...,now i want to bring judgement balancing into t...,,,Woman (cisgender),29,de
758,1847,37,user,\n ich sehe keine veränderung im Plot. Was ...,i do not see any change in the plot,,,Woman (cisgender),29,de
759,1849,2,user,\n I am working on the problem of reconstru...,i am working on the problem of reconstruc...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en


# Write back to Database

In [5]:
prompts.to_sql('expanded_prompts', conn, if_exists='replace', index=False)
#prompts_punct_cap.to_sql('punct_cap_prompts', conn, if_exists='replace', index=False)
conn.close()